In [1]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")

conn = sqlite3.connect(DB_PATH)

In [2]:
only_country = False

# Individuals Regions
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)

# Years
df_ind = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_ind_year = df_ind[["individual_wikidata_id", "birthyear"]].drop_duplicates()
df_ind_year = df_ind_year.dropna()
df_ind_year = df_ind_year[df_ind_year['birthyear']<=1850]


temporal_resolution = 10
df_ind_year["decade"] = df_ind_year["birthyear"].apply(lambda x: round(x / temporal_resolution) * temporal_resolution)

df_catalogs_id = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_catalogs = pd.read_sql_query("SELECT * FROM identifiers", conn)

if only_country:
    df_catalogs = df_catalogs[['identifiers_wikidata_id', 'country_name']].dropna()
    df_cat = pd.merge(df_catalogs_id, df_catalogs, on = 'identifiers_wikidata_id')
    df_cat = df_cat[['individual_wikidata_id','individual_name', 'country_name']].drop_duplicates()
    df_cat = df_cat.groupby(['individual_name', 'individual_wikidata_id'])['country_name'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
else:
    df_cat = df_catalogs_id.groupby(['individual_name', 'individual_wikidata_id'])['identifiers_wikidata_id'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
    
df_final = pd.merge(df_ind_year, df_cat, on = 'individual_wikidata_id')
df_final = pd.merge(df_final, df_ind_regions, on = ['individual_wikidata_id', 'individual_name'])
df_final.to_csv('../data/df_individuals_score.csv')

In [3]:
df_final.sort_values('score', ascending=False).sample(3)

,individual_wikidata_id,birthyear,decade,individual_name,score,region_code,region_name
275877,Q89743,1837.0,1840,August Klostermann,16,re_northwestern_europe,Northwestern Europe
1054,Q4820898,1792.0,1790,Jean-Brunon Rudd,5,re_belgium,Belgium
404414,Q707716,1791.0,1790,Friedrich Parrot,27,re_northwestern_europe,Northwestern Europe


In [7]:
df_final[(df_final['region_name']=='Italy')&(df_final['decade']<=500)].sort_values('score', ascending=False)

,individual_wikidata_id,birthyear,decade,individual_name,score,region_code,region_name
420426,Q7198,-42.0,-40,Ovid,207,re_italy,Italy
245799,Q1541,-106.0,-110,Cicero,205,re_italy,Italy
47105,Q1048,-99.0,-100,Julius Caesar,200,re_italy,Italy
151931,Q1398,-69.0,-70,Virgil,192,re_italy,Italy
128496,Q1430,121.0,120,Marcus Aurelius,187,re_italy,Italy
...,...,...,...,...,...,...,...
190126,Q948154,-50.0,-50,Spurius Metius Tarpa,1,re_italy,Italy
194842,Q18411886,-100.0,-100,Gaius Vettius Capitolinus,1,re_italy,Italy
198810,Q1241566,-50.0,-50,Quintus Sulpicius Camerinus,1,re_italy,Italy
140936,Q15616684,200.0,200,Hi[…],1,re_italy,Italy
